# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [8]:
import pandas as pd

# Cargar el conjunto de datos desde el enlace proporcionado
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

In [9]:
# Filtrar clientes con total_claim_amount mayor que 1000 y respuesta "Yes"
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]


In [10]:
# Filtrar el DataFrame original para clientes que respondieron "Yes"
df_yes = df[df['Response'] == 'Yes']

# Calcular el promedio de total_claim_amount por Policy Type y Gender
average_claim_by_policy_gender = df_yes.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index()
print(average_claim_by_policy_gender)


      Policy Type Gender  Total Claim Amount
0  Corporate Auto      F          433.738499
1  Corporate Auto      M          408.582459
2   Personal Auto      F          452.965929
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
5    Special Auto      M          429.527942


In [11]:
# Contar el número total de clientes por estado
customers_per_state = df['State'].value_counts().reset_index()
customers_per_state.columns = ['State', 'Customer Count']

# Filtrar estados con más de 500 clientes
filtered_states = customers_per_state[customers_per_state['Customer Count'] > 500]
print(filtered_states)


        State  Customer Count
0  California            3552
1      Oregon            2909
2     Arizona            1937
3      Nevada             993
4  Washington             888


In [12]:
# Agrupar por Education y Gender, y calcular los valores máximo, mínimo y mediano del Customer Lifetime Value
clv_stats = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()
print(clv_stats)


              Education Gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [13]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Convert the 'Effective To Date' to a datetime format
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])

# Extract the month from 'Effective To Date'
df['Month'] = df['Effective To Date'].dt.month


C:\Users\Usuario\AppData\Local\Temp\ipykernel_66092\1133201991.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])


In [14]:
# Group by State and Month and count the number of policies sold
policies_sold = df.groupby(['State', 'Month']).size().reset_index(name='Policies Sold')

# Pivot the table to arrange the months as columns and states as rows
pivot_table = policies_sold.pivot(index='State', columns='Month', values='Policies Sold').fillna(0)
print(pivot_table)


Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425


In [15]:
# Sum the total number of policies sold by state
state_totals = policies_sold.groupby('State')['Policies Sold'].sum().reset_index()

# Sort by the number of policies sold in descending order
top_states = state_totals.sort_values(by='Policies Sold', ascending=False).head(3)['State']

# Filter the pivot table to only include the top 3 states
top_states_pivot = pivot_table.loc[top_states]
print(top_states_pivot)


Month          1     2
State                 
California  1918  1634
Oregon      1565  1344
Arizona     1008   929


In [16]:
# Selecting relevant columns for analysis
marketing_channels = ['Response', 'Sales Channel', 'Policy', 'Renew Offer Type', 'EmploymentStatus']

# Filter customers who responded "Yes"
df_yes = df[df['Response'] == 'Yes']

# Use melt to unpivot the data
melted_df = df_yes.melt(id_vars='Response', value_vars=marketing_channels, var_name='Marketing Channel', value_name='Channel')

# Count the number of responses per channel
response_by_channel = melted_df.groupby('Channel').size().reset_index(name='Response Count')

# Calculate the response rate
response_rate = response_by_channel.copy()
response_rate['Response Rate'] = response_rate['Response Count'] / df['Response'].value_counts()['Yes']
print(response_rate)


          Channel  Response Count  Response Rate
0           Agent             742       0.506139
1          Branch             326       0.222374
2     Call Center             221       0.150750
3    Corporate L1              53       0.036153
4    Corporate L2              97       0.066166
5    Corporate L3             173       0.118008
6        Disabled              83       0.056617
7        Employed             850       0.579809
8   Medical Leave              86       0.058663
9          Offer1             665       0.453615
10         Offer2             769       0.524557
11         Offer3              32       0.021828
12    Personal L1             207       0.141201
13    Personal L2             344       0.234652
14    Personal L3             525       0.358117
15        Retired             221       0.150750
16     Special L1              13       0.008868
17     Special L2              20       0.013643
18     Special L3              34       0.023192
19     Unemployed   